# ETL - Wine Reviews

## Import Libraries

In [34]:
import pandas as pd
import numpy as np
from functions import read_csv_files, read_json_files
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os

## Read Datasets

In [2]:
directory_path = 'data/'

df_csv = read_csv_files(directory_path)
df_json = read_json_files(directory_path)

df = pd.concat([df_csv, df_json], ignore_index=True)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Explore the Dataset

In [3]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

In [4]:
df.shape

(410872, 13)

In [5]:
df.describe(include='all')

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
count,410741,410872,290207,410872.000000,379185.000000,410741,343318,161975,207454,197516,259942,410870,410872
unique,50,169430,47239,NaN,NaN,490,1332,18,19,15,118840,756,19186
top,US,"Still showing its tannins, this wine is develo...",Reserve,NaN,NaN,California,Napa Valley,Central Coast,Roger Voss,@vossroger,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Pinot Noir,Williams Selyem
freq,171405,8,6770,NaN,NaN,117002,15169,35187,51028,51028,22,40835,796
mean,NaN,NaN,NaN,88.241898,34.555615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,3.119719,39.400606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,80.000000,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,86.000000,16.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,88.000000,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,90.000000,40.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410872 entries, 0 to 410871
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                410741 non-null  object 
 1   description            410872 non-null  object 
 2   designation            290207 non-null  object 
 3   points                 410872 non-null  int64  
 4   price                  379185 non-null  float64
 5   province               410741 non-null  object 
 6   region_1               343318 non-null  object 
 7   region_2               161975 non-null  object 
 8   taster_name            207454 non-null  object 
 9   taster_twitter_handle  197516 non-null  object 
 10  title                  259942 non-null  object 
 11  variety                410870 non-null  object 
 12  winery                 410872 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 40.8+ MB


In [7]:
df.isnull().sum()

country                     131
description                   0
designation              120665
points                        0
price                     31687
province                    131
region_1                  67554
region_2                 248897
taster_name              203418
taster_twitter_handle    213356
title                    150930
variety                       2
winery                        0
dtype: int64

In [8]:
df.duplicated().sum()

193033

### Drop Duplicates

In [9]:
df.drop_duplicates(inplace=True)
df.shape

(217839, 13)

### Drop Columns 

In [10]:
df.drop(['taster_twitter_handle'], axis=1, inplace=True)

In [11]:
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


### Handle with missed values and obtain information of the columns

In [12]:
df['year'] = df['title'].str.extract(r'\b(20(?:0[0-9]|1[0-9]|2[0-3]))\b')

In [13]:
df['year'] = df['year'].fillna(0).astype(int)

In [14]:
df.head(10)

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013
7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,2012
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,2013
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012


In [15]:
df['region_1'] = df.apply(lambda row: row['title'].split('(')[-1].rstrip(')')
                          if pd.notna(row['title']) and pd.isna(row['region_1']) else row['region_1'], 
                          axis=1)

In [16]:
df[df['price'] > 100].count()

country        5566
description    5566
designation    4487
points         5566
price          5566
province       5566
region_1       5363
region_2       1630
taster_name    2558
title          3229
variety        5566
winery         5566
year           5566
dtype: int64

In [17]:
df['price'].fillna(df['price'].median(), inplace=True)

In [18]:
df['year'].describe()

count    217839.000000
mean       1052.189291
std        1004.310148
min           0.000000
25%           0.000000
50%        2005.000000
75%        2012.000000
max        2017.000000
Name: year, dtype: float64

## Load Data

In [37]:
load_dotenv()

def connection():
    return psycopg2.connect(
        host = os.getenv('HOST'),
        database = os.getenv('DATABASE'),
        user = os.getenv('USER'),
        password = os.getenv('PASSWORD')
    )

In [33]:
def get_columns(cursor, table_name: str) -> list:
    query = f"SELECT column_name \
            FROM information_schema.columns \
            WHERE table_name = '{table_name}' \
            ORDER BY ordinal_position;"

    cursor.execute(query)

    column_name = [row[0] for row in cursor.fetchall()]

    return column_name

In [35]:
def insert_rows(cursor, df: pd.DataFrame, table_name: str, columns: tuple):
    query = insert_query(table_name, columns)

    for (
        index,
        row,
    ) in df.iterrows():
        try:
            cursor.execute(query, tuple(row))
        except Exception as e:
            print(f"Error inserting row {index}: {e}")
            print("Row values:", tuple(row))
            print("Generated query:", cursor.mogrify(query, tuple(row)))
            raise e


def insert_query(table_name: str, columns: tuple):
    return sql.SQL("INSERT INTO {table} ({columns}) VALUES ({values})").format(
        table=sql.Identifier(table_name),
        columns=sql.SQL(", ").join(map(sql.Identifier, columns)),
        values=sql.SQL(", ").join(sql.Placeholder() * len(columns)),
    )

In [38]:
conn = connection()
cursor = conn.cursor()

In [40]:
table_name = "wine_reviews"
columns = get_columns(cursor, table_name)
columns

['id',
 'country',
 'description',
 'designation',
 'points',
 'price',
 'province',
 'region_1',
 'region_2',
 'taster_name',
 'title',
 'variety',
 'winery',
 'year']

In [44]:
insert_rows(cursor, df, table_name, columns[1:])

## Verify

In [45]:
query = "SELECT * FROM wine_reviews LIMIT 5;"
cursor.execute(query)

result = cursor.fetchall()

for row in result:
    print(row)

(1, 'Italy', "Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.", 'Vulkà Bianco', 87, 25.0, 'Sicily & Sardinia', 'Etna', 'NaN', 'Kerin O’Keefe', 'Nicosia 2013 Vulkà Bianco  (Etna)', 'White Blend', 'Nicosia', 2013)
(2, 'Portugal', "This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.", 'Avidagos', 87, 15.0, 'Douro', 'Douro', 'NaN', 'Roger Voss', 'Quinta dos Avidagos 2011 Avidagos Red (Douro)', 'Portuguese Red', 'Quinta dos Avidagos', 2011)
(3, 'US', 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.', 'NaN', 87, 14.0, 'Oregon', 'Willamette Valley', 'Willamette

In [46]:
cursor.close()
conn.close()